In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# импортируем классы BaseEstimator и TransformerMixin, 
# позволяющие написать собственные классы
from sklearn.base import BaseEstimator, TransformerMixin
# импортируем классы, необходимые для корректной работы
# пользовательских классов RareGrouper и RareGrouper2
from pandas import Series, DataFrame
from collections import Counter, defaultdict
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем custom_preprocessing
import custom_preprocessing as custprep
# импортируем класс StandardScaler, позволяющий
# выполнить стандартизацию
from sklearn.preprocessing import StandardScaler
# импортируем класс OneHotEncoder, позволяющий
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс FunctionTransformer, позволяющий
# задавать пользовательские функции
from sklearn.preprocessing import FunctionTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс GridSearchCV, позволяющий 
# выполнить поиск по сетке
from sklearn.model_selection import GridSearchCV
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score() для вычисления AUC-ROC
from sklearn.metrics import roc_auc_score
# импортируем класс StratifiedKFold для выполнения
# стратифицированной перекрестной проверки
from sklearn.model_selection import StratifiedKFold

# отключим предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'custom_preprocessing'

In [2]:
# увеличиваем количество выводимых столбцов
pd.set_option('display.max_columns', 60)
# загружаем набор данных
data = pd.read_csv('Data/Credit_OTP.csv', encoding='cp1251', sep=';')
# выводим первые 5 наблюдений
data.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.0,0,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.0,1,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.0,0,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.0,1,1,1,1,1,6,3,1,"1589,923333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.0,1,0,1,2,1,16,2,1,"1152,15",2230,NaN


In [3]:
# выводим информацию о количестве непропущенных 
# наблюдений в переменных и типах переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGREEMENT_RK                15223 non-null  int64  
 1   TARGET                      15223 non-null  int64  
 2   AGE                         15223 non-null  int64  
 3   SOCSTATUS_WORK_FL           15223 non-null  int64  
 4   SOCSTATUS_PENS_FL           15223 non-null  int64  
 5   GENDER                      15223 non-null  int64  
 6   CHILD_TOTAL                 15223 non-null  int64  
 7   DEPENDANTS                  15223 non-null  int64  
 8   EDUCATION                   15223 non-null  object 
 9   MARITAL_STATUS              15223 non-null  object 
 10  GEN_INDUSTRY                13856 non-null  object 
 11  GEN_TITLE                   13856 non-null  object 
 12  ORG_TP_STATE                13856 non-null  object 
 13  ORG_TP_FCAPITAL             138

In [4]:
# переменные, неверно записанные как количественные,
# преобразуем в тип object
object_columns = data.columns[data.columns.str.contains('FL|GENDER|OWN_AUTO|AGREEMENT')]
data[object_columns] = data[object_columns].astype('object')

# в указанных переменных заменяем запятую на точку в качестве
# десятичного разделителя и преобразуем в тип float
for col in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 
            'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
    data[col] = data[col].str.replace(',', '.').astype('float')
    
# вновь смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGREEMENT_RK                15223 non-null  object 
 1   TARGET                      15223 non-null  int64  
 2   AGE                         15223 non-null  int64  
 3   SOCSTATUS_WORK_FL           15223 non-null  object 
 4   SOCSTATUS_PENS_FL           15223 non-null  object 
 5   GENDER                      15223 non-null  object 
 6   CHILD_TOTAL                 15223 non-null  int64  
 7   DEPENDANTS                  15223 non-null  int64  
 8   EDUCATION                   15223 non-null  object 
 9   MARITAL_STATUS              15223 non-null  object 
 10  GEN_INDUSTRY                13856 non-null  object 
 11  GEN_TITLE                   13856 non-null  object 
 12  ORG_TP_STATE                13856 non-null  object 
 13  ORG_TP_FCAPITAL             138

In [5]:
# создаем список из переменных GEN_ INDUSTRY, GEN_TITLE, 
# ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR
work_cols = data.columns[data.columns.str.contains('GEN_I|GEN_T|JOB|ORG_')].tolist()

# заменяем пропуски в переменных GEN_ INDUSTRY, GEN_TITLE, 
# ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR  на "Не указано",
# если в интересующей нас переменной есть пропуск 
# И при этом переменная SOCSTATUS_PENS_FL имеет значение 1
for col in work_cols:
    data[col] = np.where((data[col].isnull()) & (data['SOCSTATUS_PENS_FL'] == 1),
                         'Не указано', data[col])
    
# заменяем пропуски в переменных GEN_ INDUSTRY, GEN_TITLE, 
# ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR  модами,
# если в интересующей нас переменной есть пропуск 
# И при этом переменная SOCSTATUS_PENS_FL имеет значение 0
for col in work_cols:
    data[col] = np.where((data[col].isnull()) & (data['SOCSTATUS_PENS_FL'] == 0),
                         data[col].value_counts().index[0], data[col])

# заменяем пропуски в переменной REGION_NM модой
data['REGION_NM'] = np.where(data['REGION_NM'].isnull(), 
                             data['REGION_NM'].value_counts().index[0], 
                             data['REGION_NM'])

# пропуски в переменной TP_PROVINCE заменим значением
# переменной FACT_ADDRESS_PROVINCE
data['TP_PROVINCE'] = np.where(data['TP_PROVINCE'].isnull(), 
                               data['FACT_ADDRESS_PROVINCE'], 
                               data['TP_PROVINCE'])
    
# заменяем пропуски в переменной 
# PREVIOUS_CARD_NUM_UTILIZED нулями
data['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(data['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, 
                                              data['PREVIOUS_CARD_NUM_UTILIZED'])

In [6]:
# выводим статистики по категориальным переменным,
# смотрим unique - количество уникальных значений,
# выявляем бесполезные переменные - переменные, у
# которых уникальных значений столько, сколько
# наблюдений (AGREEMENT_RK), и переменные с одним
# уникальным значением (DL_DOCUMENT_FL)
data.select_dtypes(include='object').describe()

,AGREEMENT_RK,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL
count,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223
unique,15223,2,2,2,7,5,32,13,6,3,11,5,81,81,80,72,11,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2
top,64585727,1,0,1,Среднее специальное,Состою в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,Кемеровская область,Краснодарский край,Краснодарский край,Краснодарский край,ЮЖНЫЙ,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1
freq,1,13847,13176,9964,6518,9416,2386,7011,6524,13689,11453,7077,674,674,674,790,2180,12765,14480,13493,12758,13782,10507,13453,14097,11940,15074,14930,14558,15223,8961,9311,14389,13855


In [7]:
# удаляем идентификационную переменную AGREEMENT_RK,
# потому что у нее количество уникальных значений
# равно количеству наблюдений, переменную DL_DOCUMENT_FL, 
# потому что у нее одно уникальное значение
data.drop(['AGREEMENT_RK', 'DL_DOCUMENT_FL'], axis=1, inplace=True)

In [8]:
# заново выделим список категориальных переменных
# и пробежим по нему, выводя частоты категорий
# по каждой категориальной переменной, чтобы
# выявить редкие категории
for col in data.select_dtypes(include='object').columns:
    print(data[col].value_counts(dropna=False))

1    13847
0     1376
Name: SOCSTATUS_WORK_FL, dtype: int64
0    13176
1     2047
Name: SOCSTATUS_PENS_FL, dtype: int64
1    9964
0    5259
Name: GENDER, dtype: int64
Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: EDUCATION, dtype: int64
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: MARITAL_STATUS, dtype: int64
Торговля                                     2386
Другие сферы                                 1709
Не указано                                   1366
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт 

Создание класса для укрупнения редких категорий

In [ ]:
class CombineRareCategories(BaseEstimator, TransformerMixin):
    """
    Параметры:
    comb_size: float, по-умолчанию 0.01
              Пороговое значение доли категории в общем количестве значений,
              меньше которого происходит объединение. По-умолчанию 1%.
    label: str, по-умолчанию "Другое"
          Метка для обобщенной категории.
    dropna: bool, по-умолчанию True
           Игнорировать пропуски.
    copy: bool, по-умолчанию True
         Возвращает копию.
    """
    def __init__(self, comb_size=0.01, label="Другое", dropna=True, copy=True):
        self.comb_size = comb_size
        self.label = label
        self.dropna = dropna
        self.copy = copy

    def _get_categories(self, xx):
        lst = []
        vc = xx.value_counts(dropna=self.dropna)
        for i in range(vc.size):
            if (vc.values[i] / xx.size) < self.comb_size:
                lst.append(vc.index[i])
        return lst

    def fit(self, X, y=None):
        self._encoder_dict = {}

        for col in X.columns:
            self._encoder_dict[col] = self._get_categories(X[col])

        return self

    def transform(self, X):
        if self.copy:
            X = X.copy()

        for col in X.columns:
            if col in self._encoder_dict:
                for i in self._encoder_dict[col]:
                    X.loc[i, col] = self.label

        return X